In [104]:
import torch 
import torch.nn as nn 
import pandas as pd 
from  torch.utils.data import Dataset, DataLoader 
from sklearn.preprocessing import MinMaxScaler 



In [155]:

# Define a simple MLP model
class MultivariateMLP(nn.Module):
    def __init__(self, num_series, num_past, hidden_size, num_future):
        super(MultivariateMLP, self).__init__()
        print ("dimension of model num_series, num_past, hidden_size,  num_future: ",num_series, num_past, hidden_size,  num_future)
        self.fc1 = nn.Linear(num_series * num_past, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_series * num_future)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = x.view(batch_size, -1, 24)  # Reshape to (batch_size, num_series, num_future)

        return x


In [214]:
import torch
import torch.nn as nn
''' a 7x7 matrix is added to model the relationship between different time series 
to be tested: instead of 7x7 matrix, how can we in addition use same matric for lag, i.e. 96x96 which might 
appear to be similar to attention. 
Once tested: a combination of things can be tried and kept in encoder-decoder structure. '''
class MultivariateMLP_Parameters(nn.Module):
    def __init__(self, num_series, num_past, hidden_size, num_future):
        super(MultivariateMLP_Parameters, self).__init__()
        self.num_series = num_series
        self.num_past = num_past
        self.num_future = num_future

        # Relationship parameters: matrix of size [num_series, num_series]
        # Each entry [i, j] represents the influence of series j on series i
        self.relationship_params = nn.Parameter(torch.randn(num_series, num_series))


        # MLP layers for prediction
        self.fc1 = nn.Linear(num_series * num_past, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_series * num_future)

    def forward(self, x):
        batch_size = x.size(0)

        # Apply relationship parameters
        # Reshape x to [batch_size * num_past, num_series]
        # Then multiply with relationship matrix
        '''
        x = x.view(-1, self.num_series)
        x = torch.matmul(x, self.relationship_params)
        '''
        
        
        relationship_weights = torch.tanh(self.relationship_params)
        # Example of applying the relationship weights to the input data
        # Assuming x is of shape [batch_size, num_past, num_series]
        # Adjust this according to your actual data shape and modeling approach
        weighted_x = torch.einsum('ij, bkj -> bki', relationship_weights, x)

        # Reshape and pass through MLP layers
        #x = weighted_x.view(batch_size, -1)  # Flatten the output
        x = weighted_x.reshape(batch_size, -1)  # Using reshape instead of view

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        # Reshape to [batch_size, num_future, num_series]
        return x.view(batch_size, self.num_future, num_series)



In [215]:
import torch
import torch.nn as nn
''' first the data is passed to Linear layer; each slice is 
data for 7 time series, this will enable to learn dependencies 
between time series. '''
class MultivariateMLP_InterSeries(nn.Module):
    def __init__(self, num_series, num_past, hidden_size, num_future):
        super(MultivariateMLP, self).__init__()
        self.num_series = num_series
        self.num_past = num_past
        self.num_future = num_future
        self.hidden_size = hidden_size

        # Inter-series layer to learn interactions between different series
        self.inter_series_layer = nn.Linear(num_series, hidden_size)

        # MLP layers for prediction
        self.fc1 = nn.Linear(hidden_size * num_past, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_series * num_future)

    def forward(self, x):
        batch_size = x.size(0)

        # Applying inter-series layer for each time step
        x = x.view(-1, self.num_series)  # Reshape for linear layer: [batch_size * num_past, num_series]
        x = self.inter_series_layer(x)  # Apply inter-series layer
        x = self.relu(x)  # Activation

        # Reshape and pass through MLP layers
        x = x.view(batch_size, -1)  # Flatten the output: [batch_size, hidden_size * num_past]
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        # Reshape to [batch_size, num_future, num_series]
        return x.view(batch_size, self.num_future, self.num_series)



In [216]:
# Custom dataset for multivariate time series
class MultivariateTimeSeriesDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
    
    def __len__(self):
        print ("length of dequence: ", len(self.sequences))
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

# Function to create sequences and targets for multivariate data
def create_multivariate_sequences(input_data, num_past, num_future):
    sequences, targets = [], []
    for i in range(len(input_data) - num_past - num_future + 1):
        seq = input_data[i:i+num_past]
        target = input_data[i+num_past:i+num_past+num_future]
        sequences.append(seq)
        targets.append(target)
    return torch.FloatTensor(sequences), torch.FloatTensor(targets)


In [217]:
dataset="../../../JupyterNotebooks/Comparison_Multivariate/dataset/ETTh1.csv"

In [218]:

# Read the CSV file
df = pd.read_csv(dataset)

# Normalize the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:,1:].values)


# Create sequences and targets
num_series = df.shape[1]-1  # Number of series (columns)
num_past = 96
num_future = 24
sequences, targets = create_multivariate_sequences(df_scaled, num_past, num_future)


In [219]:
sequences.shape

torch.Size([17301, 96, 7])

In [220]:

# Split the data into training and testing sets
train_size = int(len(sequences) * 0.8)
train_sequences, train_targets = sequences[:train_size], targets[:train_size]
test_sequences, test_targets = sequences[train_size:], targets[train_size:]

# Create datasets and dataloaders
train_dataset = MultivariateTimeSeriesDataset(train_sequences, train_targets)
test_dataset = MultivariateTimeSeriesDataset(test_sequences, test_targets)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=True)


length of dequence:  13840
length of dequence:  13840


In [221]:

# Instantiate the model
#model = MultivariateMLP(num_series, num_past, hidden_size=128, num_future=num_future)
model = MultivariateMLP_Parameters(num_series, num_past, hidden_size=128, num_future=num_future)


# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [222]:

# Training the model
epochs = 10
for epoch in range(epochs):
    print ("epoch # ", epoch)
    for seq, targets in train_loader:
        optimizer.zero_grad()
        print ("inside train_loader loop",seq.shape)
        outputs = model(seq)
        loss = criterion(outputs, targets)
        #outputs = model(seq)
        #loss = criterion(outputs.permute(0,2,1), targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')



epoch #  0
length of dequence:  13840
length of dequence:  13840
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
inside train_loader loop torch.Size([32, 96, 7])
insi

In [226]:
# Assuming 'model' is your trained model instance
optimized_params = model.relationship_params.data
print("Optimized Relationship Parameters:\n", optimized_params)


Optimized Relationship Parameters:
 tensor([[ 0.7025, -1.1348,  0.0670,  0.4848, -0.6551,  0.0051, -0.0589],
        [-0.1544,  1.7449, -0.6655,  0.1679, -0.1368, -0.4698,  0.4624],
        [-0.5275, -0.5026, -0.5956, -1.7206, -0.0857,  0.7825,  0.1924],
        [ 0.2845, -0.5702, -0.1735, -0.0979,  0.2270, -1.0953, -2.0906],
        [-0.5979, -1.2393,  1.0124, -0.0808, -0.8510,  0.5997,  0.9449],
        [ 0.2196, -1.9865,  0.2060, -0.1780,  0.0292, -0.0470,  1.2459],
        [-0.4118, -0.5027,  0.3168,  0.2994,  1.2092, -2.4794,  0.6871]])


In [224]:

# Evaluate the model
model.eval()
with torch.no_grad():
    total_loss = 0
    for seq, targets in test_loader:
        outputs = model(seq)
        print ("shapes: ", outputs.shape, targets.shape)
        loss = criterion(outputs, targets)
        #loss = criterion(outputs.permute(0,2,1), targets)

        total_loss += loss.item()
    print(f'Test Loss: {total_loss / len(test_loader)}')
    

length of dequence:  3461
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shapes:  torch.Size([32, 24, 7]) torch.Size([32, 24, 7])
shape

In [225]:
import torch
from torchviz import make_dot


In [203]:
import torch
from torchviz import make_dot

# Assuming your model class and parameters
model = MultivariateMLP_Parameters(num_series=7, num_past=96, hidden_size=64, num_future=24)

# Create a dummy input corresponding to the input size
dummy_input = torch.randn(32, 96, 7)  # Adjust the shape according to your model

# Perform a forward pass (to build the computational graph)
output = model(dummy_input)

# Create the visualization
dot = make_dot(output, params=dict(model.named_parameters()))

# Save or render the graph
dot.render('model_visualization', format='png')


'model_visualization.png'

# Import the static data for M5

In [252]:
import numpy as np
static_ =  np.load("M5_static.npy") 
print ("statis data shape: ",static_.shape)
for i in range(1,static_.shape[1] ):
    print (np.unique(static_[:,i]).shape)
    

statis data shape:  (30490, 6)
(7,)
(3,)
(10,)
(3,)
(6757,)


In [253]:
static_.shape

(30490, 6)

## Notes about the static dataset 
- The static dataset we have ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
- id is unique for each row, so that can be removed, no two ids are same and hence it is not useful to extract any information. 
- The dataset consist of 3 states 
- 10 store id 
- 3 category id: Food, hobbies,  household 
- 7 department id ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3']